In [1]:
!pip install pandas
!pip install singlestoredb
!pip install openai
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.1/340.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


#Project 4

In [29]:
import pandas as pd
import singlestoredb as db
from openai import OpenAI
import os
from PyPDF2 import PdfReader

os.environ["OPENAI_API_KEY"] = "YOUR-API-KEY"
client = OpenAI()

# Initialize OpenAI API with your API key

# Function to generate embeddings using OpenAI's API
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

def connector():
    return db.connect(host='svc-3482219c-a389-4079-b18b-d50662524e8a-shared-dml.aws-virginia-6.svc.singlestore.com',
                      port='3333', user='piyush-fcc10',
                      password='YtvjWTNaFO5AhzAiQKCdh9jqmeZSCliC', database='vectordb01')

def read_vectors(vector):
    output = []
    try:
        mydb = connector()
        mycursor = mydb.cursor()
        # sql = "SELECT product_name, review_title, review_text, dot_product(json_array_pack(%s), vector) as score, JSON_ARRAY_UNPACK(vector) as vector FROM myvectors order by score desc limit 5"
        sql = "SELECT resume_data, dot_product(json_array_pack(%s), vector) as score, JSON_ARRAY_UNPACK(vector) as vector FROM myvectorspdf order by score desc limit 5"
        mycursor.execute(sql, str(vector))
        result = mycursor.fetchall()
        for (resume_data, score, vector) in result:
            output.append({
                'resume_data': resume_data,
                'score': score
            })
        mydb.close()
    except Exception as e:
        print(e)
    return output

def insert_vector(resume_data, vector):
    try:
        mydb = connector()
        mycursor = mydb.cursor()
        sql = 'INSERT INTO myvectorspdf (resume_data, vector) VALUES (%s, JSON_ARRAY_PACK(%s))'
        mycursor.execute(sql, (resume_data, str(vector)))
        mydb.commit()
        id = mycursor.lastrowid
    except Exception as e:
        print(e)
        id = None
    finally:
        try:
            mydb.close()
        except:
            pass
    return id

# Function to read PDF and extract text
def read_pdf(file_path):
    text = ""
    with open(file_path, "rb") as f:
        pdf_reader = PdfReader(f)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

# Manual Embedding
def create_embeddings(text):
    #for row in text.iterrows():
    embedding = get_embedding(text)
    # Assuming you want to insert the entire document as a single embedding
    insert_vector(text,embedding)
    return embedding

def read_and_store_to_db():
    text = read_pdf("Piyush_resume.pdf")
    # Assuming you have a function to extract relevant data from the PDF and convert it into DataFrame format
    # Example: df = extract_data_from_pdf(data)
    create_embeddings(text)

def search_vector_from_db():
    data = "dombivli"
    vector = get_embedding(data)
    print(read_vectors(vector))

if __name__ == "__main__":

    # Following function will read PDF and store data with vectors
    read_and_store_to_db()

    # Following function will help to search the nearest review using vectors
    search_vector_from_db()


[{'resume_data': 'Piyush Rathod  \n122 Chapins street, Binghamton, 13905  \n+1607 6010579  \nObjective: To obtain a position as a Student Worker at Binghamton University – Library. I have excellent \ncommunication skills, willingness to learn new things and have experience in providing exceptional \nsupport in fast paced environment.  \nSkills:   \n● Excellent Team Leader  \n● Good Communication Skills  \n● Can work and think calmly under pressure  \n● Enjoy interacting with new people  \n● Reliable person who’s always punctual  \n● Goal Oriented  \nExperience:  \nAl Fresco Dombivli, India                    \nStudent Worker                   July 2019 – February 2020  \n• Taking Orders forms Customers.  \n• Preparing Sandwiches,  Pizzas,  Burgers,  Salads on Order.  \n•Providing Customer Service to 500 customers daily  \n• Ensuring Cleanliness and Standards  \n• Ensuring high level of customer satisfaction  \n \nNayak Tutorials,  Dombivli, India  \nFront Desk Attendant  (Part Time)   